In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=1000)
L=LaurentSeriesRing(QQ,'x', default_prec=1000)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=1000)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def census(target,lis):
    answer=0
    for n in lis:
        if n==target:answer=answer+1
    return answer

def positions(target,lst):
    ln=len(lst)
    answer=[]
    for j in [0..ln-1]:
        entry=lst[j]
        if entry==target:answer=answer+[j+1]
    return(answer)

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def JpowerConstantTerm(m,power):
    trun=J(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer
    
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay")

okay


In [8]:
for n in [1..5]:
    print((n,sigma(n,k=11)))

(1, 1)
(2, 2049)
(3, 177148)
(4, 4196353)
(5, 48828126)


In [39]:
ps=positions(0,od);print(ps) #A037453 Positive numbers whose base-5 representation contains no 3 or 4.

[1, 2, 5, 6, 7, 10, 11, 12, 25, 26, 27, 30, 31, 32, 35, 36, 37, 50, 51, 52, 55, 56, 57, 60, 61, 62, 125, 126, 127, 130, 131, 132, 135, 136, 137, 150, 151, 152, 155, 156, 157, 160, 161, 162, 175, 176, 177, 180, 181, 182, 185, 186, 187]


In [3]:
# verify narrow version, p = 7
import time

def p7RamanujanCongruenceSeries(n): 
    ans=0
    for a in [1..n+1]:
        term=rmnjntau(a)
        ans=ans+term*x^a
    return D(ans)

def recipP7(n):
    ans=L(1/p7RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def p7RecipPowerConstantTerm(k):
    g=recipP7(k)^k
    g=g.O(k+1)
    #print("g:");print(g);print()
    #print("list:")
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=2010)

no=[]
zeros=[]
od=[]
start=time.time()
for a in [1..20]: #default precision must be > max a.
    c=p7RecipPowerConstantTerm(a)
    o7=order(7,c)
    if o7==0:zeros=zeros+[a]
    od=od+[o7]
    c5=census(5,a.digits(7))
    c6=census(6,a.digits(7))
    if c5==0:
        if c6==0:
              no=no+[a]
    finish=time.time()
    print((a,finish-start))
print();print("od:");print(od);print()
ps=positions(0,od)
vno=vector(no);vps=vector(ps)
print("vno-vps:");print();print(vno-vps)

(1, 0.016301870346069336)
(2, 0.02754807472229004)
(3, 0.03902888298034668)
(4, 0.05137014389038086)
(5, 0.06168413162231445)
(6, 0.07700300216674805)
(7, 0.09350109100341797)
(8, 0.10787010192871094)
(9, 0.12219405174255371)
(10, 0.1364121437072754)
(11, 0.15051698684692383)
(12, 0.16493988037109375)
(13, 0.17883896827697754)
(14, 0.1934361457824707)
(15, 0.2087719440460205)
(16, 0.22402191162109375)
(17, 0.23960494995117188)
(18, 0.25364112854003906)
(19, 0.2673161029815674)
(20, 0.2813260555267334)

od:
[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 1, 2]

vno-vps:

(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)


In [4]:
# verify narrow version, p = 7
import time

def p7RamanujanCongruenceSeries(n): 
    ans=0
    for a in [1..n+1]:
        term=rmnjntau(a)
        ans=ans+term*x^a
    return D(ans)

def recipP7(n):
    ans=L(1/p7RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def p7RecipPowerConstantTerm(k):
    g=recipP7(k)^k
    g=g.O(k+1)
    #print("g:");print(g);print()
    #print("list:")
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=2010)

no=[]
zeros=[]
od=[]
start=time.time()
for a in [1..5000]: #default precision must be > max a.
    c=p7RecipPowerConstantTerm(a)
    o7=order(7,c)
    if o7==0:zeros=zeros+[a]
    od=od+[o7]
    c5=census(5,a.digits(7))
    c6=census(6,a.digits(7))
    if c5==0:
        if c6==0:
              no=no+[a]
    finish=time.time()
    print((a,finish-start))
print();print("od:");print(od);print()
ps=positions(0,od)
vno=vector(no);vps=vector(ps)
print("vno-vps:");print();print(vno-vps)

(1, 0.010297060012817383)
(2, 0.019703149795532227)
(3, 0.029798269271850586)
(4, 0.04095792770385742)
(5, 0.05116128921508789)
(6, 0.0651242733001709)
(7, 0.08031225204467773)
(8, 0.0949561595916748)
(9, 0.10976099967956543)
(10, 0.12374210357666016)
(11, 0.13788533210754395)
(12, 0.15237212181091309)
(13, 0.16626214981079102)
(14, 0.22179722785949707)
(15, 0.23583626747131348)
(16, 0.2501649856567383)
(17, 0.26576995849609375)
(18, 0.27979421615600586)
(19, 0.29384708404541016)
(20, 0.30794715881347656)
(21, 0.32424116134643555)
(22, 0.33988308906555176)
(23, 0.3556632995605469)
(24, 0.3719370365142822)
(25, 0.3899972438812256)
(26, 0.4082162380218506)
(27, 0.4274141788482666)
(28, 0.4460451602935791)
(29, 0.4663841724395752)
(30, 0.48548412322998047)
(31, 0.5049641132354736)
(32, 0.5248551368713379)
(33, 0.544471025466919)
(34, 0.5647580623626709)
(35, 0.5855340957641602)
(36, 0.609321117401123)
(37, 0.632317304611206)
(38, 0.6546142101287842)
(39, 0.678184986114502)
(40, 0.70236515

(327, 110.7995662689209)
(328, 111.80003929138184)
(329, 112.80091333389282)
(330, 113.81872630119324)
(331, 114.82644605636597)
(332, 115.85095930099487)
(333, 116.85779929161072)
(334, 117.87367415428162)
(335, 118.87506628036499)
(336, 119.91406416893005)
(337, 120.96681499481201)
(338, 122.02057719230652)
(339, 123.09017515182495)
(340, 124.13983511924744)
(341, 125.18726205825806)
(342, 126.24349427223206)
(343, 127.3072521686554)
(344, 128.3652422428131)
(345, 129.43749928474426)
(346, 130.51346731185913)
(347, 131.57973718643188)
(348, 132.6627812385559)
(349, 133.7536482810974)
(350, 134.84945511817932)
(351, 135.985276222229)
(352, 137.1340982913971)
(353, 138.27948713302612)
(354, 139.44856810569763)
(355, 140.6220932006836)
(356, 141.76548624038696)
(357, 142.92662024497986)
(358, 144.0934190750122)
(359, 145.23949909210205)
(360, 146.41728901863098)
(361, 147.59028100967407)
(362, 148.83511924743652)
(363, 150.05378198623657)
(364, 151.29220032691956)
(365, 152.533715009689

(651, 854.0175361633301)
(652, 857.8951852321625)
(653, 861.7971603870392)
(654, 865.689239025116)
(655, 869.6171283721924)
(656, 873.501216173172)
(657, 877.4161012172699)
(658, 881.3630232810974)
(659, 885.3205461502075)
(660, 889.273490190506)
(661, 893.2468581199646)
(662, 897.2175643444061)
(663, 901.22154712677)
(664, 905.2762780189514)
(665, 909.2904641628265)
(666, 913.3225049972534)
(667, 917.3821842670441)
(668, 921.4565403461456)
(669, 925.5318551063538)
(670, 929.6158893108368)
(671, 933.7468810081482)
(672, 937.8634660243988)
(673, 941.9818203449249)
(674, 946.1002511978149)
(675, 950.2460081577301)
(676, 954.441159248352)
(677, 958.6200811862946)
(678, 962.8166692256927)
(679, 967.0250890254974)
(680, 971.2350950241089)
(681, 975.4725151062012)
(682, 979.7188432216644)
(683, 983.9830241203308)
(684, 988.2404391765594)
(685, 992.5190691947937)
(686, 996.8085501194)
(687, 1001.0869309902191)
(688, 1005.3616812229156)
(689, 1009.6830163002014)
(690, 1014.0427069664001)
(691,

(972, 2793.4322533607483)
(973, 2802.15806722641)
(974, 2810.7443583011627)
(975, 2819.7881932258606)
(976, 2828.56583404541)
(977, 2837.5542562007904)
(978, 2846.3800320625305)
(979, 2855.117061138153)
(980, 2864.009194135666)
(981, 2872.915060043335)
(982, 2881.4826822280884)
(983, 2890.118910074234)
(984, 2898.6739630699158)
(985, 2907.2898240089417)
(986, 2915.9313480854034)
(987, 2924.6126232147217)
(988, 2933.2830452919006)
(989, 2941.9897260665894)
(990, 2950.709342956543)
(991, 2959.426063299179)
(992, 2968.16685628891)
(993, 2976.9402751922607)
(994, 2985.685622215271)
(995, 2994.474603176117)
(996, 3003.338730096817)
(997, 3012.161950111389)
(998, 3021.0360839366913)
(999, 3029.9554471969604)
(1000, 3038.813669204712)
(1001, 3047.693132162094)
(1002, 3056.559740304947)
(1003, 3065.4901192188263)
(1004, 3074.505998134613)
(1005, 3083.4961111545563)
(1006, 3092.5236101150513)
(1007, 3101.5876982212067)
(1008, 3110.704243183136)
(1009, 3119.789954185486)
(1010, 3128.883764982223

(1287, 6408.720649957657)
(1288, 6423.445315122604)
(1289, 6438.30896115303)
(1290, 6453.154319286346)
(1291, 6467.908016204834)
(1292, 6482.763743162155)
(1293, 6497.59840631485)
(1294, 6512.508093357086)
(1295, 6527.328784227371)
(1296, 6542.240888118744)
(1297, 6557.1424951553345)
(1298, 6572.063184261322)
(1299, 6587.040867090225)
(1300, 6602.078143119812)
(1301, 6617.172429084778)
(1302, 6632.328908205032)
(1303, 6647.41467499733)
(1304, 6662.444580078125)
(1305, 6677.608043193817)
(1306, 6692.681891202927)
(1307, 6707.830229997635)
(1308, 6723.016872167587)
(1309, 6738.287078142166)
(1310, 6753.596741199493)
(1311, 6768.82488822937)
(1312, 6783.989675045013)
(1313, 6799.305563211441)
(1314, 6814.7257771492)
(1315, 6830.072703123093)
(1316, 6845.507501125336)
(1317, 6860.844786167145)
(1318, 6876.207396268845)
(1319, 6891.603209018707)
(1320, 6907.153438091278)
(1321, 6922.610835075378)
(1322, 6938.164800167084)
(1323, 6953.797665119171)
(1324, 6969.282253026962)
(1325, 6984.82940

(1600, 12387.870309114456)
(1601, 12411.497891187668)
(1602, 12434.577649354935)
(1603, 12458.127029180527)
(1604, 12481.937990188599)
(1605, 12505.933331012726)
(1606, 12530.070415258408)
(1607, 12553.795069217682)
(1608, 12577.674133062363)
(1609, 12601.892320156097)
(1610, 12625.861491203308)
(1611, 12649.588020086288)
(1612, 12673.458998203278)
(1613, 12697.632320165634)
(1614, 12721.719281196594)
(1615, 12745.990056991577)
(1616, 12770.15551328659)
(1617, 12794.177090168)
(1618, 12818.177513122559)
(1619, 12842.612126111984)
(1620, 12867.3523812294)
(1621, 12892.076818943024)
(1622, 12916.427962303162)
(1623, 12941.095842123032)
(1624, 12965.494229078293)
(1625, 12990.07746720314)
(1626, 13014.529759168625)
(1627, 13038.602810144424)
(1628, 13062.954820156097)
(1629, 13087.409263134003)
(1630, 13111.93012714386)
(1631, 13136.706611394882)
(1632, 13161.210233211517)
(1633, 13186.105384349823)
(1634, 13211.490463018417)
(1635, 13235.842190265656)
(1636, 13260.084485292435)
(1637, 13

KeyboardInterrupt: 

In [5]:
print();print("od:");print(od);print()
ps=positions(0,od)
vno=vector(no);vps=vector(ps)
print("vno-vps:");print();print(vno-vps)


od:
[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 1, 3, 2, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 1, 1, 1, 1, 2, 3, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 3, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 2, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 3, 3, 1, 1, 1, 1, 4, 2, 3, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2, 2, 3, 2, 3, 3, 3, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 3, 2, 1, 1, 1, 1, 2, 2, 4, 1, 1, 1, 1, 3, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3

In [2]:
# verify narrow version, p = 7
import time
p=7
def p7RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 290
    ans=0
    for a in [1..n+1]:
        term=a*sigma(a,k=3)
        ans=ans+term*x^a
    return D(ans)

def recipP7(n):
    ans=L(1/p7RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def p7RecipPowerConstantTerm(k):
    g=recipP7(k)^k
    g=g.O(k+1)
    #print("g:");print(g);print()
    #print("list:")
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    
bound=50
L=LaurentSeriesRing(QQ,'x', default_prec=bound+10)

no=[]
zeros=[]
od=[]
start=time.time()
for a in [1..bound]: #default precision must be > max a.
    c=p7RecipPowerConstantTerm(a)
    o=order(p,c)
    if o==0:
        zeros=zeros+[a]
        adp=a.digits(p)
        c5=census(5,adp)
        c6=census(6,adp)
        if c5>0:
            no=no+[a]
        if c6>0:
            no=no+[a]
        
    finish=time.time()
    print((a,adp,finish-start))
print();print(no)

(1, [1], 0.016505002975463867)
(2, [2], 0.016955137252807617)
(3, [3], 0.01729297637939453)
(4, [4], 0.01776409149169922)
(5, [4], 0.018343210220336914)
(6, [4], 0.01873302459716797)
(7, [0, 1], 0.019172191619873047)
(8, [1, 1], 0.019806861877441406)
(9, [2, 1], 0.02024698257446289)
(10, [3, 1], 0.020843029022216797)
(11, [4, 1], 0.021308183670043945)
(12, [4, 1], 0.021934986114501953)
(13, [4, 1], 0.022433996200561523)
(14, [0, 2], 0.023047208786010742)
(15, [1, 2], 0.02358388900756836)
(16, [2, 2], 0.0242311954498291)
(17, [3, 2], 0.024801969528198242)
(18, [4, 2], 0.025495052337646484)
(19, [4, 2], 0.026112079620361328)
(20, [4, 2], 0.026845216751098633)
(21, [0, 3], 0.027493000030517578)
(22, [1, 3], 0.02827000617980957)
(23, [2, 3], 0.028974056243896484)
(24, [3, 3], 0.0297698974609375)
(25, [4, 3], 0.030495882034301758)
(26, [4, 3], 0.031545162200927734)
(27, [4, 3], 0.03243303298950195)
(28, [0, 4], 0.03320002555847168)
(29, [1, 4], 0.03420686721801758)
(30, [2, 4], 0.0351901054

In [4]:
# verify narrow version, p = 7
import time
p=7
def p7RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 290
    ans=0
    for a in [1..n+1]:
        term=a*sigma(a,k=3)
        ans=ans+term*x^a
    return D(ans)

def recipP7(n):
    ans=L(1/p7RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def p7RecipPowerConstantTerm(k):
    g=recipP7(k)^k
    g=g.O(k+1)
    #print("g:");print(g);print()
    #print("list:")
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    
bound=5000
L=LaurentSeriesRing(QQ,'x', default_prec=bound+10)

no=[]
zeros=[]
od=[]
start=time.time()
for a in [1..bound]: #default precision must be > max a.
    c=p7RecipPowerConstantTerm(a)
    o=order(p,c)
    if o==0:
        zeros=zeros+[a]
        adp=a.digits(p)
        c5=census(5,adp)
        c6=census(6,adp)
        if c5>0:
            no=no+[a]
        if c6>0:
            no=no+[a]
        
        finish=time.time()
        print((a,adp,finish-start))
print();print(no)

(1, [1], 0.04494786262512207)
(2, [2], 0.09104776382446289)
(3, [3], 0.15803194046020508)
(4, [4], 0.23691582679748535)
(7, [0, 1], 0.6469109058380127)
(8, [1, 1], 0.8236327171325684)
(9, [2, 1], 0.9979069232940674)
(10, [3, 1], 1.179426908493042)
(11, [4, 1], 1.3561620712280273)
(14, [0, 2], 1.8781049251556396)
(15, [1, 2], 2.0513498783111572)
(16, [2, 2], 2.231433868408203)
(17, [3, 2], 2.403265953063965)
(18, [4, 2], 2.577523946762085)
(21, [0, 3], 3.118821144104004)
(22, [1, 3], 3.297078847885132)
(23, [2, 3], 3.4756438732147217)
(24, [3, 3], 3.6915059089660645)
(25, [4, 3], 3.868161916732788)
(28, [0, 4], 4.391824722290039)
(29, [1, 4], 4.566593885421753)
(30, [2, 4], 4.743760824203491)
(31, [3, 4], 4.916239976882935)
(32, [4, 4], 5.090651035308838)
(49, [0, 0, 1], 8.108319997787476)
(50, [1, 0, 1], 8.321002960205078)
(51, [2, 0, 1], 8.504166841506958)
(52, [3, 0, 1], 8.683082818984985)
(53, [4, 0, 1], 8.863856792449951)
(56, [0, 1, 1], 9.41254997253418)
(57, [1, 1, 1], 9.59965205

(521, [3, 4, 3, 1], 109.36896514892578)
(522, [4, 4, 3, 1], 109.62766575813293)
(539, [0, 0, 4, 1], 114.131826877594)
(540, [1, 0, 4, 1], 114.3974609375)
(541, [2, 0, 4, 1], 114.67067003250122)
(542, [3, 0, 4, 1], 114.94343900680542)
(543, [4, 0, 4, 1], 115.22617602348328)
(546, [0, 1, 4, 1], 116.0146107673645)
(547, [1, 1, 4, 1], 116.28310084342957)
(548, [2, 1, 4, 1], 116.55078482627869)
(549, [3, 1, 4, 1], 116.82153010368347)
(550, [4, 1, 4, 1], 117.12886190414429)
(553, [0, 2, 4, 1], 117.96578884124756)
(554, [1, 2, 4, 1], 118.24892902374268)
(555, [2, 2, 4, 1], 118.53755903244019)
(556, [3, 2, 4, 1], 118.82252788543701)
(557, [4, 2, 4, 1], 119.10952496528625)
(560, [0, 3, 4, 1], 119.94665694236755)
(561, [1, 3, 4, 1], 120.22224569320679)
(562, [2, 3, 4, 1], 120.49993014335632)
(563, [3, 3, 4, 1], 120.78390693664551)
(564, [4, 3, 4, 1], 121.05588793754578)
(567, [0, 4, 4, 1], 121.89144086837769)
(568, [1, 4, 4, 1], 122.16305613517761)
(569, [2, 4, 4, 1], 122.4497971534729)
(570, [3

(1131, [4, 0, 2, 3], 377.0097839832306)
(1134, [0, 1, 2, 3], 379.22055292129517)
(1135, [1, 1, 2, 3], 380.01156282424927)
(1136, [2, 1, 2, 3], 380.6841058731079)
(1137, [3, 1, 2, 3], 381.3826539516449)
(1138, [4, 1, 2, 3], 382.0712878704071)
(1141, [0, 2, 2, 3], 384.3738269805908)
(1142, [1, 2, 2, 3], 385.1339750289917)
(1143, [2, 2, 2, 3], 385.9179949760437)
(1144, [3, 2, 2, 3], 386.6368067264557)
(1145, [4, 2, 2, 3], 387.39578104019165)
(1148, [0, 3, 2, 3], 389.6151428222656)
(1149, [1, 3, 2, 3], 390.3835999965668)
(1150, [2, 3, 2, 3], 391.18798303604126)
(1151, [3, 3, 2, 3], 392.007239818573)
(1152, [4, 3, 2, 3], 392.6052188873291)
(1155, [0, 4, 2, 3], 394.52981185913086)
(1156, [1, 4, 2, 3], 395.16682386398315)
(1157, [2, 4, 2, 3], 395.838898897171)
(1158, [3, 4, 2, 3], 396.511944770813)
(1159, [4, 4, 2, 3], 397.2161650657654)
(1176, [0, 0, 3, 3], 408.76352095603943)
(1177, [1, 0, 3, 3], 409.47555589675903)
(1178, [2, 0, 3, 3], 410.1803698539734)
(1179, [3, 0, 3, 3], 410.9246549606

KeyboardInterrupt: 

In [5]:
print(no)

[]


In [2]:
import time
p=7
def p7RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 290
    ans=0
    for a in [1..n+1]:
        term=a*sigma(a,k=3)
        ans=ans+term*x^a
    return D(ans)

def recipP7(n):
    ans=L(1/p7RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def p7RecipPowerConstantTerm(k):
    g=recipP7(k)^k
    g=g.O(k+1)
    #print("g:");print(g);print()
    #print("list:")
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    
bound=1600
L=LaurentSeriesRing(QQ,'x', default_prec=bound+10)

no=[]
zeros=[]
od=[]
start=time.time()
for a in [1..bound]: #default precision must be > max a.
    c=p7RecipPowerConstantTerm(a)
    o=order(p,c)
    adp=a.digits(p)
    c5=census(5,adp)
    c6=census(6,adp)
    if (c5==0)&(c6==0)&(o!=0):
        no=no+[a]
    print((a,len(no)))
print();print(no)

(1, 0)
(2, 0)
(3, 0)
(4, 0)
(5, 0)
(6, 0)
(7, 0)
(8, 0)
(9, 0)
(10, 0)
(11, 0)
(12, 0)
(13, 0)
(14, 0)
(15, 0)
(16, 0)
(17, 0)
(18, 0)
(19, 0)
(20, 0)
(21, 0)
(22, 0)
(23, 0)
(24, 0)
(25, 0)
(26, 0)
(27, 0)
(28, 0)
(29, 0)
(30, 0)
(31, 0)
(32, 0)
(33, 0)
(34, 0)
(35, 0)
(36, 0)
(37, 0)
(38, 0)
(39, 0)
(40, 0)
(41, 0)
(42, 0)
(43, 0)
(44, 0)
(45, 0)
(46, 0)
(47, 0)
(48, 0)
(49, 0)
(50, 0)
(51, 0)
(52, 0)
(53, 0)
(54, 0)
(55, 0)
(56, 0)
(57, 0)
(58, 0)
(59, 0)
(60, 0)
(61, 0)
(62, 0)
(63, 0)
(64, 0)
(65, 0)
(66, 0)
(67, 0)
(68, 0)
(69, 0)
(70, 0)
(71, 0)
(72, 0)
(73, 0)
(74, 0)
(75, 0)
(76, 0)
(77, 0)
(78, 0)
(79, 0)
(80, 0)
(81, 0)
(82, 0)
(83, 0)
(84, 0)
(85, 0)
(86, 0)
(87, 0)
(88, 0)
(89, 0)
(90, 0)
(91, 0)
(92, 0)
(93, 0)
(94, 0)
(95, 0)
(96, 0)
(97, 0)
(98, 0)
(99, 0)
(100, 0)
(101, 0)
(102, 0)
(103, 0)
(104, 0)
(105, 0)
(106, 0)
(107, 0)
(108, 0)
(109, 0)
(110, 0)
(111, 0)
(112, 0)
(113, 0)
(114, 0)
(115, 0)
(116, 0)
(117, 0)
(118, 0)
(119, 0)
(120, 0)
(121, 0)
(122, 0)
(123, 0)
(

(925, 0)
(926, 0)
(927, 0)
(928, 0)
(929, 0)
(930, 0)
(931, 0)
(932, 0)
(933, 0)
(934, 0)
(935, 0)
(936, 0)
(937, 0)
(938, 0)
(939, 0)
(940, 0)
(941, 0)
(942, 0)
(943, 0)
(944, 0)
(945, 0)
(946, 0)
(947, 0)
(948, 0)
(949, 0)
(950, 0)
(951, 0)
(952, 0)
(953, 0)
(954, 0)
(955, 0)
(956, 0)
(957, 0)
(958, 0)
(959, 0)
(960, 0)
(961, 0)
(962, 0)
(963, 0)
(964, 0)
(965, 0)
(966, 0)
(967, 0)
(968, 0)
(969, 0)
(970, 0)
(971, 0)
(972, 0)
(973, 0)
(974, 0)
(975, 0)
(976, 0)
(977, 0)
(978, 0)
(979, 0)
(980, 0)
(981, 0)
(982, 0)
(983, 0)
(984, 0)
(985, 0)
(986, 0)
(987, 0)
(988, 0)
(989, 0)
(990, 0)
(991, 0)
(992, 0)
(993, 0)
(994, 0)
(995, 0)
(996, 0)
(997, 0)
(998, 0)
(999, 0)
(1000, 0)
(1001, 0)
(1002, 0)
(1003, 0)
(1004, 0)
(1005, 0)
(1006, 0)
(1007, 0)
(1008, 0)
(1009, 0)
(1010, 0)
(1011, 0)
(1012, 0)
(1013, 0)
(1014, 0)
(1015, 0)
(1016, 0)
(1017, 0)
(1018, 0)
(1019, 0)
(1020, 0)
(1021, 0)
(1022, 0)
(1023, 0)
(1024, 0)
(1025, 0)
(1026, 0)
(1027, 0)
(1028, 0)
(1029, 0)
(1030, 0)
(1031, 0)
(1032